In [7]:
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import datetime
from pathlib import Path


def fetch_fedfunds(start: str, end: str) -> pd.DataFrame:
    """Fetch monthly Effective Federal Funds Rate (FEDFUNDS) from FRED."""
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    df = DataReader("FEDFUNDS", "fred", start_dt, end_dt)
    df = df.rename(columns={"FEDFUNDS": "fedfunds"})
    df.index.name = "date"

    # Debug prints
    print(f"\n📊 Retrieved FEDFUNDS data from {start} to {end}")
    print(f"Rows: {len(df)}, Columns: {list(df.columns)}")
    print("First 5 rows:")
    print(df.head())
    print("Last 5 rows:")
    print(df.tail())
    return df


def to_quarterly(df: pd.DataFrame, method: str = "mean") -> pd.DataFrame:
    """Convert monthly FEDFUNDS to quarterly frequency.

    method: 'mean' (quarterly average) or 'last' (end-of-quarter)
    """
    if method == "last":
        q = df.resample("Q").last()
    else:
        q = df.resample("Q").mean()
    q.index.name = "quarter"

    # Debug prints
    print(f"\n📈 Converted to quarterly data using method='{method}'")
    print(f"Rows: {len(q)}, Columns: {list(q.columns)}")
    print("First 5 rows:")
    print(q.head())
    print("Last 5 rows:")
    print(q.tail())
    return q
  
start = "2010-01-01"
end = datetime.today().strftime("%Y-%m-%d")

monthly = fetch_fedfunds(start, end)
quarterly = to_quarterly(monthly, method="mean")



📊 Retrieved FEDFUNDS data from 2010-01-01 to 2025-08-25
Rows: 187, Columns: ['fedfunds']
First 5 rows:
            fedfunds
date                
2010-01-01      0.11
2010-02-01      0.13
2010-03-01      0.16
2010-04-01      0.20
2010-05-01      0.20
Last 5 rows:
            fedfunds
date                
2025-03-01      4.33
2025-04-01      4.33
2025-05-01      4.33
2025-06-01      4.33
2025-07-01      4.33

📈 Converted to quarterly data using method='mean'
Rows: 63, Columns: ['fedfunds']
First 5 rows:
            fedfunds
quarter             
2010-03-31  0.133333
2010-06-30  0.193333
2010-09-30  0.186667
2010-12-31  0.186667
2011-03-31  0.156667
Last 5 rows:
            fedfunds
quarter             
2024-09-30  5.263333
2024-12-31  4.650000
2025-03-31  4.330000
2025-06-30  4.330000
2025-09-30  4.330000


In [ ]:
import os
import pandas as pd
import pandasdmx

# Mapping of country codes (example emerging markets).
EM_COUNTRIES = {
    "BR": "Brazil",
    "IN": "India",
    "MX": "Mexico",
    "ZA": "South Africa",
    "TR": "Turkey",
}

# helper to explore IMF DOTS metadata before fetching data
def explore_dots_structure():
    req = pandasdmx.Request("IMF")
    resp = req.datastructure("DOT")
    dsd = resp.datastructure["DOT"]

    print("\n📊 IMF DOTS Dimensions:")
    for dim in dsd.dimensions:
        print(f"- {dim.id} ({dim.local_repr.enumerated} values)")


# Fetch monthly exports data for a given country
def fetch_exports(country_code: str, start: str, end: str) -> pd.DataFrame:
    """Fetch monthly exports data for a given country from IMF DOTS (partner=W00 = World)."""
    try:
        print("running")
        req = pandasdmx.Request("IMF")
        key = f"M.{country_code}.W00.TXG_FOB_USD"
        data = req.data(
            resource_id="DOT", key=key, params={"startPeriod": start, "endPeriod": end}
        )
        df = data.to_pandas().reset_index()
        df = df.rename(columns={"value": "exports_usd"})
        df["country"] = country_code
        df = df[["TIME_PERIOD", "country", "exports_usd"]]

        print(
            f"\n📊 Retrieved exports for {country_code} ({EM_COUNTRIES.get(country_code, '')})"
        )
        print(f"Rows: {len(df)}, Columns: {list(df.columns)}")
        print("First 5 rows:")
        print(df.head())
        print("end")
        return df
    except Exception as e:
        print(f"❌ Error fetching {country_code}: {e}")
        return pd.DataFrame()


def to_quarterly(df: pd.DataFrame) -> pd.DataFrame:
    """Convert monthly exports to quarterly sums."""
    df["TIME_PERIOD"] = pd.to_datetime(df["TIME_PERIOD"])
    df = df.set_index("TIME_PERIOD")
    q = df.groupby("country").resample("Q")["exports_usd"].sum().reset_index()
    q = q.rename(columns={"TIME_PERIOD": "quarter"})

    # Debug print
    print("\n📈 Converted to quarterly exports")
    print(f"Rows: {len(q)}, Columns: {list(q.columns)}")
    print("First 5 rows:")
    print(q.head())

    return q

start = "2015-01"
end = "2023-12"

all_data = []
for code in EM_COUNTRIES.keys():
    df = fetch_exports(code, start, end)
    if not df.empty:
        all_data.append(df)

if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    q_all = to_quarterly(df_all)

    os.makedirs("data", exist_ok=True)
    df_all.to_csv("data/exports_monthly_raw.csv", index=False)
    q_all.to_csv("data/exports_quarterly.csv", index=False)

    print("\n✅ Saved CSV files:")
    print(" - data/exports_monthly_raw.csv")
    print(" - data/exports_quarterly.csv")
else:
    print("⚠️ No data fetched. Check country codes and IMF DOTS availability.")

running
❌ Error fetching BR: <Resource.data: 'data'> endpoint at https://sdmxcentral.imf.org/ws/public/sdmxapi/rest/data/DOT/M.BR.W00.TXG_FOB_USD?startPeriod=2015-01&endPeriod=2023-12
running
❌ Error fetching IN: <Resource.data: 'data'> endpoint at https://sdmxcentral.imf.org/ws/public/sdmxapi/rest/data/DOT/M.IN.W00.TXG_FOB_USD?startPeriod=2015-01&endPeriod=2023-12
running
❌ Error fetching MX: <Resource.data: 'data'> endpoint at https://sdmxcentral.imf.org/ws/public/sdmxapi/rest/data/DOT/M.MX.W00.TXG_FOB_USD?startPeriod=2015-01&endPeriod=2023-12
running
❌ Error fetching ZA: <Resource.data: 'data'> endpoint at https://sdmxcentral.imf.org/ws/public/sdmxapi/rest/data/DOT/M.ZA.W00.TXG_FOB_USD?startPeriod=2015-01&endPeriod=2023-12
running
❌ Error fetching TR: <Resource.data: 'data'> endpoint at https://sdmxcentral.imf.org/ws/public/sdmxapi/rest/data/DOT/M.TR.W00.TXG_FOB_USD?startPeriod=2015-01&endPeriod=2023-12
⚠️ No data fetched. Check country codes and IMF DOTS availability.
